import

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Device

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [15]:
in_channels = 3
num_classes = 3
learning_rate = 0.001
batch_size = 4
num_epochs = 5

Load Custom Data

In [16]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

#1.Configuration
root_dir = r"C:\Users\ninad\Desktop\Deep Learning\Custom Dataset (Image)\cats_dogs_resized"
image_size = (224,224)
train_test_split_ratio = 0.8

#2.Define Transforms

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

#3.Load Dataset from Folder
dataset = datasets.ImageFolder(root=root_dir, transform=transform)

#Print Classes
print("Class Name: ", dataset.classes)
print("Class to index mapping: ", dataset.class_to_idx)

#5.Split Dataset
train_size = int(train_test_split_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

#6.Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for images, labels in train_loader:
    print("Train batch - images shape:", images.shape)  # [4, 3, 224, 224]
    print("Train batch - labels:", labels)              # tensor([0, 1, ...])
    break

for images, labels in test_loader:
    print("Test batch - images shape:", images.shape)
    print("Test batch - labels:", labels)
    break

Class Name:  ['cats', 'dogs']
Class to index mapping:  {'cats': 0, 'dogs': 1}
Train batch - images shape: torch.Size([4, 3, 224, 224])
Train batch - labels: tensor([0, 1, 1, 0])
Test batch - images shape: torch.Size([4, 3, 224, 224])
Test batch - labels: tensor([1, 0, 0, 1])


Create Fully Connected Network

In [23]:
class CNN(nn.Module):
    def __init__(self, in_channels=3,num_classes=2):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32 ,kernel_size=3, stride=1, padding=1), #32x112x112
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #32x56x56
        )

        self.flattened_size = 32*56*56

        self.fc = nn.Sequential(
            nn.Linear(in_features=self.flattened_size, out_features=128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

Save and Load function

Initialize Model

In [24]:
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device=device)

Loss and Optimizer

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    losses = []
        
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data and target to the correct device (CPU or GPU)
        data = data.to(device)
        target = target.to(device)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, target)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training information
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{total_step}], Loss: {loss.item():.4f}")

Epoch [1/5], Step [1/3], Loss: 1.2072
Epoch [1/5], Step [2/3], Loss: 9.0139
Epoch [1/5], Step [3/3], Loss: 3.0769
Epoch [2/5], Step [1/3], Loss: 2.0821
Epoch [2/5], Step [2/3], Loss: 0.7111
Epoch [2/5], Step [3/3], Loss: 0.5237
Epoch [3/5], Step [1/3], Loss: 0.5370
Epoch [3/5], Step [2/3], Loss: 0.9993
Epoch [3/5], Step [3/3], Loss: 0.5220
Epoch [4/5], Step [1/3], Loss: 0.4702
Epoch [4/5], Step [2/3], Loss: 0.4267
Epoch [4/5], Step [3/3], Loss: 0.3850
Epoch [5/5], Step [1/3], Loss: 0.4106
Epoch [5/5], Step [2/3], Loss: 0.4048
Epoch [5/5], Step [3/3], Loss: 0.3370


In [31]:
def evaluate(loader, model, device):
    # Custom label to detect if it's train or test loader
    is_train = hasattr(loader.dataset, 'dataset') and loader.dataset.dataset == train_dataset

    print("Checking accuracy on {} data".format("training" if is_train else "test"))

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, preds = scores.max(1)

            num_correct += (preds == y).sum().item()  # .item() to get scalar
            num_samples += preds.size(0)

    accuracy = 100.0 * num_correct / num_samples
    print(f'Accuracy: {num_correct}/{num_samples} = {accuracy:.2f}%')

    model.train()
    return accuracy

In [32]:
evaluate(train_loader, model, device)
evaluate(test_loader, model, device)

Checking accuracy on test data
Accuracy: 12/12 = 100.00%
Checking accuracy on test data
Accuracy: 3/4 = 75.00%


75.0